In [1]:
import torch
import torch.optim as optim
from torchvision import datasets, transforms
from torchvision.models.mobilenet import mobilenet_v2
from torch.optim.lr_scheduler import StepLR
from torch.nn import CrossEntropyLoss
import pandas as pd
import os
import numpy as np
import pandas as pd
from PIL import Image
from torch.utils.data import DataLoader, Dataset
from scipy.special import softmax

In [2]:
def train(model, device, train_loader, optimizer, epoch):
    log_interval = 10
    loss_func = CrossEntropyLoss()
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_func(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader), loss.item()))

In [3]:
def test_model(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    loss_func = CrossEntropyLoss()
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss_func(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [4]:
class DatasetPeople(Dataset):

    def __init__(self, file_path, root_path, transform=None):
        self.data = pd.read_csv(file_path)
        self.root_path = root_path
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        image_path = os.path.join(self.root_path, self.data.loc[index, 'id'])
        image = Image.open(image_path)
        label = self.data.loc[index, 'target_people']

        if self.transform is not None:
            image = self.transform(image)

        return image, label

In [13]:
batch_size = 200
learning_rate = 1.0
reduce_lr_gamma = 0.7
epochs = 7
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Device: {} Epochs: {} Batch size: {}'.format(device, epochs, batch_size))



kwargs = {'batch_size': batch_size}
if torch.cuda.is_available():
    kwargs.update({'num_workers': 1, 'pin_memory': True})

transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
])


dataset1 = DatasetPeople('valid.csv', 'valid', transform=transform)
dataset2 = DatasetPeople('train.csv', 'train', transform=transform)
dataset3 = DatasetPeople('sample_submission.csv', 'test', transform=transform)
print('Length train: {} Length test: {}'.format(len(dataset1), len(dataset2)))

valid_loader = torch.utils.data.DataLoader(dataset1, **kwargs)
train_loader = torch.utils.data.DataLoader(dataset2, **kwargs)
test_loader = torch.utils.data.DataLoader(dataset3, **kwargs)
print('Number of train batches: {} Number of test batches: {}'.format(len(train_loader), len(test_loader)))



model = mobilenet_v2(pretrained=True)
model.classifier[1] = torch.nn.Linear(in_features=model.classifier[1].in_features, out_features=10)
model.to(device)
optimizer = optim.Adadelta(model.parameters(), lr=learning_rate)


scheduler = StepLR(optimizer, step_size=1, gamma=reduce_lr_gamma)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test_model(model, device, valid_loader)
    scheduler.step()

torch.save(model.state_dict(), "mnist_cnn.pt")


predictions = []

for data, target in test_loader:
    data = data.to(device)
    output = model(data)
    pred = output.softmax(dim=1)
    predictions += list(pred.cpu().detach().numpy()[:, 1])

    

fin_res = pd.read_csv('sample_submission.csv')
fin_res['target_people'] = predictions
fin_res.to_csv('./submission.csv', index=False)

Device: cpu Epochs: 7 Batch size: 200
Length train: 1536 Length test: 9003
Number of train batches: 46 Number of test batches: 23


D:\jupyter\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\jupyter\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MobileNet_V2_Weights.IMAGENET1K_V1`. You can also use `weights=MobileNet_V2_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Train Epoch: 1 [0/9003 (0%)]	Loss: 2.185494
Train Epoch: 1 [2000/9003 (22%)]	Loss: 1.049719
Train Epoch: 1 [4000/9003 (43%)]	Loss: 0.461148
Train Epoch: 1 [6000/9003 (65%)]	Loss: 0.596276
Train Epoch: 1 [8000/9003 (87%)]	Loss: 0.448452

Test set: Average loss: 0.9228, Accuracy: 1092/1536 (71%)

Train Epoch: 2 [0/9003 (0%)]	Loss: 0.780264
Train Epoch: 2 [2000/9003 (22%)]	Loss: 0.439289
Train Epoch: 2 [4000/9003 (43%)]	Loss: 0.387061
Train Epoch: 2 [6000/9003 (65%)]	Loss: 0.406281
Train Epoch: 2 [8000/9003 (87%)]	Loss: 0.299207

Test set: Average loss: 0.9209, Accuracy: 1041/1536 (68%)

Train Epoch: 3 [0/9003 (0%)]	Loss: 0.527543
Train Epoch: 3 [2000/9003 (22%)]	Loss: 0.279136
Train Epoch: 3 [4000/9003 (43%)]	Loss: 0.247355
Train Epoch: 3 [6000/9003 (65%)]	Loss: 0.225600
Train Epoch: 3 [8000/9003 (87%)]	Loss: 0.151852

Test set: Average loss: 0.9698, Accuracy: 1151/1536 (75%)

Train Epoch: 4 [0/9003 (0%)]	Loss: 0.458054
Train Epoch: 4 [2000/9003 (22%)]	Loss: 0.157226
Train Epoch: 4 [4000